<a href="https://colab.research.google.com/github/gigajet/transformer/blob/master/running_fairseq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preparations

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Google Colab doesn't support `ln`.

Mounted at /content/drive


In [ ]:
# Install fairseq
!git clone https://github.com/pytorch/fairseq
%cd fairseq
!pip install --editable ./

Cloning into 'fairseq'...
remote: Enumerating objects: 31434, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 31434 (delta 23), reused 38 (delta 19), pack-reused 31381
Receiving objects: 100% (31434/31434), 21.69 MiB | 29.49 MiB/s, done.
Resolving deltas: 100% (23124/23124), done.
/content/fairseq
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/fairseq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 236 kB 4.8 MB/s 
     |████████████████████████████████| 123 kB 67.1 MB/s 
     |████████████████████████████████| 92 kB 14.1 MB/s 
     |████████████████████████████████| 112 kB 70.0 MB/s 
     |████████████████████████████████| 596 kB 64.8 MB/s 
  Created wheel for antlr4-python3-runtime: 

In [ ]:
# Dependencies for preprocessing
!pip install fastBPE sacremoses subword_nmt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 880 kB 8.2 MB/s 
  Created wheel for fastBPE: filename=fastBPE-0.1.0-cp37-cp37m-linux_x86_64.whl size=483891 sha256=431d235c765c175e942608eb14730dd5c62f79f726b1fb7bb257c8dbc56a207f
  Stored in directory: /root/.cache/pip/wheels/bd/d4/0e/0d317a65f77d3f8049fedd8a2ee0519164cf3e6bd77ef886f1
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=2a35353a0259740da3671351820f2032f9ca4d884f2bfb0239f3308f061913aa
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built fastBPE sacremoses


In [ ]:
# Run this if you get error "importlib_metadata.PackageNotFoundError: No package metadata was found for fairseq" in next cell
import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"

## Verify that fairseq is now usable (optional)

In [ ]:
# This cell is for verifying the install
# Interactive translation
import torch
import fairseq

# List available models
torch.hub.list('pytorch/fairseq')  # [..., 'transformer.wmt16.en-de', ... ]

# Load a transformer trained on WMT'16 En-De
# Note: WMT'19 models use fastBPE instead of subword_nmt, see instructions below
en2de = torch.hub.load('pytorch/fairseq', 'transformer.wmt16.en-de',
                       tokenizer='moses', bpe='subword_nmt')
en2de.eval()  # disable dropout

# The underlying model is available under the *models* attribute
assert isinstance(en2de.models[0], fairseq.models.transformer.TransformerModel)

# Move model to GPU for faster translation
# en2de.cuda()

# Translate a sentence
en2de.translate('Hello world!')
# 'Hallo Welt!'

# Batched translation
en2de.translate(['Hello world!', 'The cat sat on the mat.'])
# ['Hallo Welt!', 'Die Katze saß auf der Matte.']

Using cache found in /root/.cache/torch/hub/pytorch_fairseq_main
Using cache found in /root/.cache/torch/hub/pytorch_fairseq_main
2022-05-27 16:01:51 | INFO | fairseq.file_utils | https://dl.fbaipublicfiles.com/fairseq/models/wmt16.en-de.joined-dict.transformer.tar.bz2 not found in cache, downloading to /tmp/tmptx7b08c0

 11%|█▏        | 249775104/2193287384 [00:10<03:49, 8459560.90B/s]


KeyboardInterrupt: ignored

## Prepare & Train IWSLT'14 German to English (Transformer)

In [ ]:
# Download and prepare the data
%cd /content/fairseq/examples/translation/
!bash prepare-iwslt14.sh
%cd ../..

/content/fairseq/examples/translation
Cloning Moses github repository (for tokenization scripts)...
Cloning into 'mosesdecoder'...
remote: Enumerating objects: 148097, done.
remote: Counting objects: 100% (525/525), done.
remote: Compressing objects: 100% (229/229), done.
remote: Total 148097 (delta 323), reused 441 (delta 292), pack-reused 147572
Receiving objects: 100% (148097/148097), 129.88 MiB | 23.60 MiB/s, done.
Resolving deltas: 100% (114349/114349), done.
Cloning Subword NMT repository (for BPE pre-processing)...
Cloning into 'subword-nmt'...
remote: Enumerating objects: 590, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 590 (delta 3), reused 4 (delta 1), pack-reused 576
Receiving objects: 100% (590/590), 245.03 KiB | 5.00 MiB/s, done.
Resolving deltas: 100% (352/352), done.
--2022-05-24 15:05:27--  http://dl.fbaipublicfiles.com/fairseq/data/iwslt14/de-en.tgz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfil

In [ ]:
# Preprocess/binarize the data
# Look below on how to set environment variable in GG Colab
%env TEXT=examples/translation/iwslt14.tokenized.de-en
!fairseq-preprocess --source-lang de --target-lang en \
    --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test \
    --destdir /content/drive/MyDrive/translation/iwslt14.tokenized.de-en \
    --workers 20


env: TEXT=examples/translation/iwslt14.tokenized.de-en
/bin/bash: fairseq-preprocess: command not found


In [ ]:
# Train the model, please change --max-epoch depends whether you use gpu/cpu
%env CUDA_VISIBLE_DEVICES=0 
!fairseq-train \
    /content/drive/MyDrive/translation/iwslt14.tokenized.de-en \
    --arch transformer_iwslt_de_en --share-decoder-input-output-embed \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-epoch 15 \
    --max-tokens 4096 \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
    --save-dir /content/drive/MyDrive/translation/iwslt14.tokenized.de-en/checkpoints


env: CUDA_VISIBLE_DEVICES=0
/bin/bash: fairseq-train: command not found


In [ ]:
# Evaluate our model
!fairseq-generate /content/drive/MyDrive/translation/iwslt14.tokenized.de-en \
    --path /content/drive/MyDrive/translation/iwslt14.tokenized.de-en/checkpoints/checkpoint_best.pt \
    --batch-size 128 --beam 5 --remove-bpe

/bin/bash: fairseq-generate: command not found


## Prepare IWSLT'15 English - Vietnamese data
Data is already in Drive `/content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/dataset`, unnecessary

In [ ]:
!unzip /content/drive/MyDrive/translation/en-vi.zip -d /content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/dataset
# !cp /content/drive/MyDrive/translation/myprepare-iwslt15-en-vi.sh .

Archive:  /content/drive/MyDrive/translation/en-vi.zip
  inflating: /content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/dataset/IWSLT15.TED.dev2010.en-vi.en  
  inflating: /content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/dataset/IWSLT15.TED.dev2010.en-vi.vi  
  inflating: /content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/dataset/IWSLT15.TED.tst2010.en-vi.en  
  inflating: /content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/dataset/IWSLT15.TED.tst2010.en-vi.vi  
  inflating: /content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/dataset/IWSLT15.TED.tst2011.en-vi.en  
  inflating: /content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/dataset/IWSLT15.TED.tst2011.en-vi.vi  
  inflating: /content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/dataset/IWSLT15.TED.tst2012.en-vi.en  
  inflating: /content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/dataset/IWSLT15.TED.tst2012.en-vi.vi  
  inflating: /content/drive/MyDrive/translation/iwslt15.t

In [ ]:
%cd /content/mymodel/models
!mkdir iwslt15.tokenized.en-vi
!unzip /content/drive/MyDrive/translation/en-vi.zip -d iwslt15.tokenized.en-vi/tmp
!sh prepare-iwslt15en2vi.sh
# After this, we have `train valid test` in iwslt15.tokenized.en-vi

/content/mymodel/models
Archive:  /content/drive/MyDrive/translation/en-vi.zip
  inflating: iwslt15.tokenized.en-vi/tmp/IWSLT15.TED.dev2010.en-vi.en  
  inflating: iwslt15.tokenized.en-vi/tmp/IWSLT15.TED.dev2010.en-vi.vi  
  inflating: iwslt15.tokenized.en-vi/tmp/IWSLT15.TED.tst2010.en-vi.en  
  inflating: iwslt15.tokenized.en-vi/tmp/IWSLT15.TED.tst2010.en-vi.vi  
  inflating: iwslt15.tokenized.en-vi/tmp/IWSLT15.TED.tst2011.en-vi.en  
  inflating: iwslt15.tokenized.en-vi/tmp/IWSLT15.TED.tst2011.en-vi.vi  
  inflating: iwslt15.tokenized.en-vi/tmp/IWSLT15.TED.tst2012.en-vi.en  
  inflating: iwslt15.tokenized.en-vi/tmp/IWSLT15.TED.tst2012.en-vi.vi  
  inflating: iwslt15.tokenized.en-vi/tmp/IWSLT15.TED.tst2013.en-vi.en  
  inflating: iwslt15.tokenized.en-vi/tmp/IWSLT15.TED.tst2013.en-vi.vi  
  inflating: iwslt15.tokenized.en-vi/tmp/IWSLT15.TED.tst2015.en-vi.en  
  inflating: iwslt15.tokenized.en-vi/tmp/IWSLT15.TED.tst2015.en-vi.vi  
  inflating: iwslt15.tokenized.en-vi/tmp/train.en  
  inf

In [ ]:
# OPTIONAL
!sh apply-bpe-split-test-set.sh

apply_bpe.py to test.00-10.en...
apply_bpe.py to test.11-20.en...
apply_bpe.py to test.21-30.en...
apply_bpe.py to test.31-40.en...
apply_bpe.py to test.41-.en...
apply_bpe.py to test.00-10.vi...
apply_bpe.py to test.11-20.vi...
apply_bpe.py to test.21-30.vi...
apply_bpe.py to test.31-40.vi...
apply_bpe.py to test.41-.vi...


In [ ]:
# Preprocess/binarize the data
# Look below on how to set environment variable in GG Colab
# THIS CELL ASSUME SPLITTED DATASET IS UPLOADED, I DONT KNOW WTF IT IS. I WILL FORGET IT ANYWAY.
%env TEXT=iwslt15.tokenized.en-vi
!fairseq-preprocess --source-lang en --target-lang vi \
    --trainpref $TEXT/train --validpref $TEXT/valid \
    --testpref $TEXT/test.00-10,$TEXT/test.11-20,$TEXT/test.21-30,$TEXT/test.31-40,$TEXT/test.41- \
    --destdir /content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/dataset \
    --workers 20
    

env: TEXT=iwslt15.tokenized.en-vi
2022-05-25 17:02:25 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-05-25 17:02:25 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='/content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/dataset', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, o

## Train IWSLT'15 English-Vietnamese (Transformer)

In [ ]:
# arch transformer, ở đây dùng transformer_iwslt_de_en là dùng lại architecture của nó
# Train the model, please change --max-epoch depends whether you use gpu/cpu
# see stop-time-hours
%env CUDA_VISIBLE_DEVICES=0 
!fairseq-train \
    /content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/dataset \
    --arch transformer_iwslt_de_en --share-decoder-input-output-embed \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-epoch 36 \
    --max-tokens 4096 \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
    --save-dir checkpoints
    # --save-dir /content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/checkpoints


env: CUDA_VISIBLE_DEVICES=0
/bin/bash: fairseq-train: command not found


In [ ]:
# Evaluate our model, change the path
!fairseq-generate /content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/dataset \
    --path /content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/checkpoints-nntransformer/checkpoint_best.pt \
    --batch-size 128 --beam 5 --remove-bpe \
    --user-dir /content/mymodel

Streaming output truncated to the last 5000 lines.
D-1655	-0.6798378825187683	Chúng ta nên vượt qua những phép đo này để bắt đầu giảm thiểu sự tổn thương : Cái mà chúng ta thải ra càng ít tổng số lượng CO2 , và do đó , nhiệt độ càng ít càng tốt .
P-1655	-0.4345 -0.1261 -0.3655 -2.0279 -0.4340 -1.0682 -0.8722 -0.0891 -0.5580 -0.2233 -0.1208 -0.0524 -0.2461 -1.1189 -1.8477 -1.3713 -0.4789 -0.5455 -3.0675 -0.6532 -0.2279 -0.0989 -2.4088 -0.1666 -1.5202 -0.0423 -0.5555 -0.3621 -0.1159 -0.5781 -0.9149 -0.2967 -1.2525 -0.2277 -0.4315 -0.1601 -0.0357 -0.0030 -0.8197 -2.4289 -0.4621 -0.3276 -0.0952
S-5475	He not only brags about my college degree , but also that I was the first woman , and that I am the first woman to drive him through the streets of Kabul .
T-5475	Ông ấy không những khoe về bằng cao đẳng của tôi , mà còn về tôi là người phụ nữ đầu tiên , và tôi là người phụ nữ đầu tiên có thể chở ông ấy trên những con đường ở Kabul .
H-5475	-0.4306568503379822	Ông ta không chỉ khoe khoang về 

## Try to add FuzzyLayer to Transformer


In [ ]:
# FuzzyLayer
import torch
import math
from torch import nn

class FuzzyLayer(nn.Module):

    def __init__(self, output_dim: int, **kwargs):
        super(FuzzyLayer, self).__init__(**kwargs)
        self.output_dim = output_dim
        self.fuzzy_degree = nn.Parameter(torch.empty(output_dim))
        self.sigma = nn.Parameter(torch.ones(output_dim))
        self.reset_parameters()
        
    def reset_parameters(self) -> None:
        # Setting a=sqrt(5) in kaiming_uniform is the same as initializing with
        # uniform(-1/sqrt(in_features), 1/sqrt(in_features)). For details, see
        # https://github.com/pytorch/pytorch/issues/57109
        nn.init.uniform_(self.fuzzy_degree)

    def forward(self, input, **kwargs):
        x = torch.repeat_interleave(torch.unsqueeze(input,-1), self.output_dim, dim=-1)
        fuzzy_out = torch.exp(
                      -torch.sum(
                          torch.square((x-self.fuzzy_degree)/(self.sigma**2))            
                          ,dim=-2, keepdims=False)
              )
        return fuzzy_out


class FuzzyRuleLayer(nn.Module):

    def __init__(self, input_dim: int, output_dim,**kwargs):
        super(FuzzyRuleLayer, self).__init__(**kwargs)
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.layers = nn.ModuleList([
            FuzzyLayer(output_dim) for _ in range(input_dim)
        ])

    def forward(self, input):
        batch_size, input_dim = input.size()
        an=torch.ones(batch_size, self.output_dim)
        for layer in self.layers:
            an=an*layer(input)
        return an

z=FuzzyRuleLayer(5,35)
y=torch.ones(3,5)
print(z(y).shape)
print(z(y))

torch.Size([3, 35])
tensor([[6.7112e-06, 6.8293e-04, 1.1360e-05, 1.4338e-05, 3.1253e-06, 4.4289e-04,
         1.2499e-04, 2.6953e-04, 5.7319e-06, 3.1216e-05, 5.5655e-05, 3.2802e-04,
         3.7528e-02, 1.1846e-05, 7.1205e-07, 2.0275e-06, 7.1357e-07, 4.7466e-04,
         1.9231e-06, 2.3387e-03, 8.4445e-06, 1.2212e-04, 1.2321e-04, 4.9719e-04,
         9.7817e-03, 7.2931e-03, 2.3511e-06, 5.0083e-02, 1.0146e-05, 1.0202e-06,
         5.4978e-04, 8.7630e-04, 9.0879e-02, 2.8873e-08, 5.0999e-03],
        [6.7112e-06, 6.8293e-04, 1.1360e-05, 1.4338e-05, 3.1253e-06, 4.4289e-04,
         1.2499e-04, 2.6953e-04, 5.7319e-06, 3.1216e-05, 5.5655e-05, 3.2802e-04,
         3.7528e-02, 1.1846e-05, 7.1205e-07, 2.0275e-06, 7.1357e-07, 4.7466e-04,
         1.9231e-06, 2.3387e-03, 8.4445e-06, 1.2212e-04, 1.2321e-04, 4.9719e-04,
         9.7817e-03, 7.2931e-03, 2.3511e-06, 5.0083e-02, 1.0146e-05, 1.0202e-06,
         5.4978e-04, 8.7630e-04, 9.0879e-02, 2.8873e-08, 5.0999e-03],
        [6.7112e-06, 6.8293e-0

## Try to add MyLSTM to fairseq (CPU only)


### Some notes
+ Follow the tutorial at https://fairseq.readthedocs.io/en/latest/tutorial_simple_lstm.html, add `@register_model` and `@register_model_architecture` to appropriate class.

+ That's not all, you must either:

1. Move your `model_name.py` to `fairseq/models`

2. Inside your directory (supposed `user_dir`), create a new folder named `models` and move your file there. In the command line (such as `fairseq-train`), specify `--user-dir /path/to/user_dir`

Because each time the command line tool runs, it must run some bootstrap code to get the current model list, parse your arguments, etc. Therefore you must follow some of their rules.

In [ ]:
# Make mymodel path and prepare my_model (in this case `MyLSTM.py`) in user_dir (in this case `mymodel`)
%cd /content
!rm -rf my_transformer
!git clone https://github.com/gigajet/transformer my_transformer
!mkdir mymodel
%cd mymodel
!mkdir models
%cd models 
!cp /content/my_transformer/MyFairseqLSTM.py .
!cp -r /content/my_transformer/layer .


/content
fatal: destination path 'my_transformer' already exists and is not an empty directory.
mkdir: cannot create directory ‘mymodel’: File exists
/content/mymodel
mkdir: cannot create directory ‘models’: File exists
/content/mymodel/models


In [ ]:
# Train the model, please change --max-epoch depends whether you use gpu/cpu
%env CUDA_VISIBLE_DEVICES=0 
!fairseq-train \
    /content/drive/MyDrive/translation/iwslt14.tokenized.de-en \
    --encoder-dropout 0.2 --decoder-dropout 0.2 \
    --optimizer adam --lr 0.005 --lr-shrink 0.5 \
    --max-epoch 1 \
    --max-tokens 12000 \
    --save-dir checkpoints \
    --user-dir /content/mymodel \
    --arch mylstm_default \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric

env: CUDA_VISIBLE_DEVICES=0
2022-04-22 10:02:58 | INFO | numexpr.utils | NumExpr defaulting to 2 threads.
2022-04-22 10:02:59 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-04-22 10:03:01 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': '/content/mymodel', 'empty_cache_freq': 0, 'all_gath

## Get mymodel from github and prepare directory structure that satisfies fairseq requirement. Adding custom model to fairseq in `/content/mymodel`

In [ ]:
# Make mymodel path and prepare my_model (in this case `MyLSTM.py`) in user_dir (in this case `mymodel`)
%cd /content
!rm -rf my_transformer
!git clone https://github.com/gigajet/transformer my_transformer
!rm -rf mymodel
!mkdir mymodel
%cd mymodel
!cp -r /content/my_transformer models

# Xóa thể loại không liên quan
%cd models
!rm -f string_reverser_*.py

# Finalize
%cd /content/fairseq


/content
Cloning into 'my_transformer'...
remote: Enumerating objects: 428, done.
remote: Counting objects: 100% (185/185), done.
remote: Compressing objects: 100% (140/140), done.
remote: Total 428 (delta 113), reused 109 (delta 45), pack-reused 243
Receiving objects: 100% (428/428), 29.87 MiB | 34.88 MiB/s, done.
Resolving deltas: 100% (248/248), done.
/content/mymodel
/content/mymodel/models
/content/fairseq


In [ ]:
# fix the import layer to mymodel.layer
# F*** PYTHON3 IMPORT
!find /content/mymodel -type f -exec sed -i "s/from layer\./from mymodel.models.layer./g" {} \;
!find /content/mymodel -type f -exec sed -i "s/from nnFairseqTransformer/from mymodel.models.nnFairseqTransformer/g" {} \;

##Training nntransformer_default

In [ ]:
# Use MyTransformer to train en-vi data
# Train the model, please change --max-epoch depends whether you use gpu/cpu
# see stop-time-hours
%env CUDA_VISIBLE_DEVICES=0 
!fairseq-train \
    /content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/dataset \
    --arch nntransformer_default \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-epoch 100 \
    --max-tokens 4096 \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
    --save-dir /content/drive/MyDrive/iwslt15.tokenized.en-vi/checkpoints-nntransformer \
    --user-dir /content/mymodel


env: CUDA_VISIBLE_DEVICES=0
2022-05-06 15:16:16 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-05-06 15:16:18 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': '/content/mymodel', 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': No

##Training custom proposal transformer
Arch is `proposalX_default`

Change save-dir accordingly

In [ ]:
# Train the model, please change --max-epoch depends whether you use gpu/cpu
# see stop-time-hours,
# change PROPOSAL env variable to choose between proposals

%env DATASET=iwslt16.tokenized.en-fr
%env CUDA_VISIBLE_DEVICES=0 
%env PROPOSAL=7
!fairseq-train \
    /content/drive/MyDrive/translation/$DATASET/dataset \
    --arch "proposal${PROPOSAL}_default" \
    --save-dir /content/drive/MyDrive/translation/$DATASET/checkpoints-proposal-$PROPOSAL \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-epoch 100 \
    --max-tokens 2048 \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
    --user-dir /content/mymodel \
    --dim-model 128 \


env: DATASET=iwslt16.tokenized.en-fr
env: CUDA_VISIBLE_DEVICES=0
env: PROPOSAL=7
2022-05-24 15:42:25 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-05-24 15:42:27 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': '/content/mymodel', 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'mo

## Preprocess the IWSLT16 en-fr dataset


In [ ]:
%cd /content/mymodel/models
# !pip install gdown
# !gdown https://drive.google.com/uc?id=1KpNH2mN5YvVIbBAXOBFX_nLEYuSfezvZ
!sh prepare-iwslt16-en-fr.sh
# After this, we have `train valid test` in iwslt15.tokenized.en-vi

/content/mymodel/models
Cloning Moses github repository (for tokenization scripts)...
fatal: destination path 'mosesdecoder' already exists and is not an empty directory.
Cloning Subword NMT repository (for BPE pre-processing)...
fatal: destination path 'subword-nmt' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Downloading...
From: https://drive.google.com/uc?id=1KpNH2mN5YvVIbBAXOBFX_nLEYuSfezvZ
To: /content/mymodel/models/orig/en-fr.tgz
100% 26.3M/26.3M [00:00<00:00, 180MB/s] 
Data successfully downloaded.
en-fr/
en-fr/IWSLT16.TED.dev2010.en-fr.en.xml
en-fr/IWSLT16.TED.dev2010.en-fr.fr.xml
en-fr/IWSLT16.TED.tst2010.en-fr.en.xml
en-fr/IWSLT16.TED.tst2010.en-fr.fr.xml
en-fr/IWSLT16.TED.tst2011.en-fr.en.xml
en-fr/IWSLT16.TED.tst2011.en-fr.fr.xml
en-fr/IWSLT16.TED.tst2012.en-fr.en.xml
en-fr/IWSLT16.TED.tst2012.en-fr.fr.xml
en-fr/IWSLT16.TED.tst2013.en-fr.en.xml
en-fr/IWSLT16.TED.tst2013.en-

In [ ]:
# Preprocess/binarize the data
# Look below on how to set environment variable in GG Colab
%env TEXT=iwslt16.tokenized.en-fr
!fairseq-preprocess --source-lang en --target-lang fr \
    --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test \
    --destdir somecrazypath \
    --workers 20

env: TEXT=iwslt16.tokenized.en-fr
2022-05-25 15:40:37 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-05-25 15:40:37 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='somecrazypath', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, p

## Fairseq inference on model


In [ ]:
from mymodel.models.proposal_9 import Proposal9Transformer
mymodel_en2vi = Proposal9Transformer.from_pretrained(
  '/content/drive/MyDrive/translation/try-running-fairseq-inference',
  checkpoint_file='proposal9-dfuzzy96-best.pt',
  data_name_or_path='/content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/dataset',
  bpe='subword_nmt',
  bpe_codes='/content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/dataset/code'
)
mymodel_en2vi.eval()
mymodel_en2vi.translate('Did you lose your half?')

ModuleNotFoundError: ignored

In [ ]:
from mymodel.models.nnFairseqTransformer import NNTransformer